<a href="https://colab.research.google.com/github/mzucali/pyMinColab/blob/main/ExtractFromSummaryEMPA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LEGGE IL FILE SUMMARY.TXT ed estrae DL e SD:

=>>input il file summary.txt o qualunque altro nome MA .TXT
- start Marzo 2023
- aggiornamento Novembre 2023

1) CARICA FILE OSSIDI

In [1]:
# load and read summary file from EMPA
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving 9-3-23.txt to 9-3-23.txt
User uploaded file "9-3-23.txt" with length 467554 bytes


clean text from unwanted lines

In [7]:
def filter_text(input_file, output_file, specific_texts):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        for line in infile:
            if not any(text in line for text in specific_texts):
                outfile.write(line)

# Replace these values with your actual file names and specific text
input_filename = fn
output_filename = 'output.txt'
specific_texts = ['Overlap	correction', '(Zn	cps)']

filter_text(input_filename, output_filename, specific_texts)

1) Separate block

In [2]:
import re
import pandas as pd

#file_path = './4-4-23.txt'  # Replace with the actual file path
#file_name = file_path.replace("./", "")
file_path = fn
file_name = file_path.replace(".txt", "")

# Initialize variables
sections = []
current_section = []

# Read the file line by line
with open(file_path, 'r') as file:
    lines = file.readlines()

# Identify and extract sections
for line in lines:
    if re.match(r'^Unknown Specimen', line):
        print("match: ", line)
        # Start of a new section
        if current_section:
            # Save the current section
            sections.append(current_section)
            current_section = []
    # Add line to the current section
    current_section.append(line.strip())

# Save the last section
if current_section:
    sections.append(current_section)

# Process each section
for section_content in sections:
    # Find the start and end indices for the desired block
    start_index = next((i for i, line in enumerate(section_content) if line.startswith('Element\tPeak(mm)')), None)
    end_index = next((i for i, line in enumerate(section_content) if line.startswith('Element\tf(chi)')), None)
    print("start =", start_index)
    print("end =", end_index)
    # Check if both start and end indices are found
    if start_index is not None and end_index is not None and end_index > start_index:
        # Extract the desired block
        block_content = section_content[start_index:end_index]

        # Extract the header dynamically from the second line
        header = block_content[0].split('\t')
        header.insert(0, "ID")

        try:
            # Create a DataFrame for the current block
            df = pd.DataFrame([re.split(r'\s+', line.strip()) for line in block_content[2:]], columns=header)

            # Print or further process the DataFrame
            print(df)
        except ValueError as e:
            print("ValueError:", e)
            print("Header:", header)
            print("Block content:", block_content)
    else:
        print("Start or end index not found or invalid in the section.")

match:  Unknown Specimen	No.	1	 

match:  Unknown Specimen	No.	2	 

match:  Unknown Specimen	No.	3	 

match:  Unknown Specimen	No.	4	 

match:  Unknown Specimen	No.	6	 

match:  Unknown Specimen	No.	8	 

match:  Unknown Specimen	No.	9	 

match:  Unknown Specimen	No.	10	 

match:  Unknown Specimen	No.	11	 

match:  Unknown Specimen	No.	12	 

match:  Unknown Specimen	No.	13	 

match:  Unknown Specimen	No.	15	 

match:  Unknown Specimen	No.	16	 

match:  Unknown Specimen	No.	17	 

match:  Unknown Specimen	No.	18	 

match:  Unknown Specimen	No.	19	 

match:  Unknown Specimen	No.	20	 

match:  Unknown Specimen	No.	21	 

match:  Unknown Specimen	No.	22	 

match:  Unknown Specimen	No.	24	 

match:  Unknown Specimen	No.	25	 

match:  Unknown Specimen	No.	27	 

match:  Unknown Specimen	No.	30	 

match:  Unknown Specimen	No.	31	 

match:  Unknown Specimen	No.	32	 

match:  Unknown Specimen	No.	33	 

match:  Unknown Specimen	No.	34	 

match:  Unknown Specimen	No.	35	 

match:  Unknown Specimen	No

2) S.D. extract

In [ ]:
import re
import pandas as pd

#file_path = './test2.txt'  # Replace with the actual file path

# Initialize variables
sections = []
current_section = []
current_comment = None  # Variable to store the value in Comment line

# Read the file line by line
with open(file_path, 'r') as file:
    lines = file.readlines()

# Identify and extract sections
for line in lines:
    if re.match(r'^Unknown Specimen', line):
        print("match: ", line)
        # Start of a new section
        if current_section:
            # Save the current section
            sections.append((current_comment, current_section))
            current_section = []
    # Add line to the current section
    current_section.append(line.strip())
    if line:
        try:
            line = line.split("\t")
            #print("line ",line)
            if(line[2] == "Comment :"):
                #print("line_s[2]",line_s[2])
                # Keep track of the comment value
                current_comment= line[3]
                # Just a spacing line
                print("comm",current_comment)
        except IndexError:
            #print("no comm")
            pass

# Save the last section
if current_section:
    sections.append((current_comment, current_section))

# Initialize a dictionary to store lists of S.D.(%) values for each element
sd_values = {}
# Initialize a list to store Comment values
comment_list = []

# Process each section
for comment, section_content in sections:
    # Find the start and end indices for the desired block
    start_index = next((i for i, line in enumerate(section_content) if line.startswith('Element\tPeak(mm)')), None)
    end_index = next((i for i, line in enumerate(section_content) if line.startswith('Element\tf(chi)')), None)
    print("start =", start_index)
    print("end =", end_index)
    # Check if both start and end indices are found
    if start_index is not None and end_index is not None and end_index > start_index:
        # Extract the desired block
        block_content = section_content[start_index:end_index]

        # Extract the header dynamically from the second line
        header = block_content[0].split('\t')
        header.insert(0, "ID")

        try:
            # Create a DataFrame for the current block
            df = pd.DataFrame([re.split(r'\s+', line.strip()) for line in block_content[1:]], columns=header)

            # Extract S.D.(%) values for each element in the current block
            sd_column_values = df.set_index('Element')['S.D.(%)'].to_dict()

            # Update the sd_values dictionary with S.D.(%) values for the current block
            for element, sd_value in sd_column_values.items():
                sd_values.setdefault(element, []).append(sd_value)

            # Print or further process the DataFrame
            print(df)
        except ValueError as e:
            print("ValueError:", e)
            print("Header:", header)
            print("Block content:", block_content)
        # Add Comment value to the list
        comment_list.append(comment)
    else:
        print("Start or end index not found or invalid in the section.")

# Print the extracted lists of S.D.(%) values for each element
print("S.D.(%) values:")
for element, sd_value_list in sd_values.items():
    print(f"{element}: {sd_value_list}")

3) S.D. save

In [ ]:
print("S.D.(%) values:")
print(sd_values)
for element, sd_value in sd_values.items():

    print(f"{element}: {sd_value}")

output_sd_file_path = file_name+'SD.txt'
df_sd = pd.DataFrame.from_dict(sd_values)
df_sd.insert(0, 'Comment', comment_list)  # Insert the Comment column at the beginning
df_sd.to_csv(output_sd_file_path, index=False, sep='\t')
print(f'S.D. values saved to {output_sd_file_path}')

4) D.L. extract

In [ ]:
import re
import pandas as pd

#file_path = './test2.txt'  # Replace with the actual file path

# Initialize variables
sections = []
current_section = []
current_comment = None  # Variable to store the value in Comment line

# Read the file line by line
with open(file_path, 'r') as file:
    lines = file.readlines()

# Identify and extract sections
for line in lines:
    if re.match(r'^Unknown Specimen', line):
        print("match: ", line)
        # Start of a new section
        if current_section:
            # Save the current section
            sections.append((current_comment, current_section))
            current_section = []
    # Add line to the current section
    current_section.append(line.strip())
    if line:
        try:
            line = line.split("\t")
            #print("line ",line)
            if(line[2] == "Comment :"):
                #print("line_s[2]",line_s[2])
                # Keep track of the comment value
                current_comment= line[3]
                # Just a spacing line
                print("comm",current_comment)
        except IndexError:
            #print("no comm")
            pass

# Save the last section
if current_section:
    sections.append((current_comment, current_section))

# Initialize a dictionary to store lists of D.L. values for each element
dl_values = {}
# Initialize a list to store Comment values
comment_list = []

# Process each section
for comment, section_content in sections:
    # Find the start and end indices for the desired block
    start_index = next((i for i, line in enumerate(section_content) if line.startswith('Element\tPeak(mm)')), None)
    end_index = next((i for i, line in enumerate(section_content) if line.startswith('Element\tf(chi)')), None)
    print("start =", start_index)
    print("end =", end_index)
    # Check if both start and end indices are found
    if start_index is not None and end_index is not None and end_index > start_index:
        # Extract the desired block
        block_content = section_content[start_index:end_index]

        # Extract the header dynamically from the second line
        header = block_content[0].split('\t')
        header.insert(0, "ID")

        try:
            # Create a DataFrame for the current block
            df = pd.DataFrame([re.split(r'\s+', line.strip()) for line in block_content[1:]], columns=header)

            # Extract S.D.(%) values for each element in the current block
            dl_column_values = df.set_index('Element')['D.L.(ppm)'].to_dict()

            # Update the dl_values dictionary with D.L.(ppm) values for the current block
            for element, dl_value in dl_column_values.items():
                dl_values.setdefault(element, []).append(dl_value)

            # Print or further process the DataFrame
            print(df)
        except ValueError as e:
            print("ValueError:", e)
            print("Header:", header)
            print("Block content:", block_content)
        # Add Comment value to the list
        comment_list.append(comment)
    else:
        print("Start or end index not found or invalid in the section.")

# Print the extracted lists of S.D.(%) values for each element
print("D.L.(ppm) values:")
for element, dl_value_list in dl_values.items():
    print(f"{element}: {dl_value_list}")

5) D.L. save file

In [ ]:
print("D.L. values:")
print(dl_values)
for element, dl_value in dl_values.items():

    print(f"{element}: {dl_value}")

output_dl_file_path = file_name+'DL.txt'
df_dl = pd.DataFrame.from_dict(dl_values)
df_dl.insert(0, 'Comment', comment_list)  # Insert the Comment column at the beginning
df_dl.to_csv(output_dl_file_path, index=False, sep='\t')
print(f'D.L. values saved to {output_dl_file_path}')

OLD

**OLD version** - Estrazione SD

In [ ]:
num_elements = input("QUANTI ELEMENTI ANALIZZATI? ")
#num_elements = 11
loop = int(num_elements)+2

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Mar 14 11:30:45 2023
GRAZIE ALESSANDRO COMUNIAN
@author: alex
"""

#file_in = open("summary.txt", "r")
file_in = open(fn,"r")
outFile = open("result2_tmpSD.txt", "w")

lines = file_in.readlines()

for i, line in enumerate(lines):
    # Just a spacing line
    #line_sp = line.split()
    #print(line_sp)
    line_s = line.split("\t")
    #print(line_s)
    if line_s:
        try:
            if(line_s[2] == "Comment :"):
                #print("line_s[2]",line_s[2])
                # Keep track of the comment value
                comm = line_s[3]
                # Just a spacing line
                #print("comm",comm)
        except IndexError:
            pass

        if(line_s[0] == "Curr.(A) :"):
#        if(line_s[0] == "Element	"):
            #print("ID {0} Comment".format(lines[i+1][:-1]))
            outFile.write("ID {0} Comment\n".format(lines[i+1][:-1]))
            for j in range(2,loop):
                #print("{0} {1}\n".format(lines[i+j][:-1], comm), end="")
                outFile.write("{0} {1}\n".format(lines[i+j][:-1], comm))
                #outFile.write(" {0}\n".format(lines))

file_in.close()
outFile.close() #result2.txt

print("completed SD extraction")

Remove spaces SD

In [ ]:
#RESULT3 con spazi e tab corretti SD
newopen = open("result2_final_tmpSD.txt", "w")
with open("result2_tmpSD.txt") as f:
  first_line = f.readline()
  first_line_split = first_line.split()
  first_line = '\t'.join(first_line_split)+'\n'
  newopen.write(first_line)
  #print(first_line)

with open("result2_tmpSD.txt") as f:
  for line in f:
    if 'ID Element	Peak(mm)' not in line:
      line_split = line.split()
      line = '\t'.join(line_split)+'\n'
      #print(line)
      newopen.write(line)
newopen.close()


input_file_path = 'result2_final_tmpSD.txt'
output_file_path = 'result2_finalSD.txt'

# Read the file and filter out empty lines
with open(input_file_path, 'r') as file:
    lines = file.readlines()
    non_empty_lines = [line.strip() for line in lines if line.strip()]

# Write non-empty lines back to the file
with open(output_file_path, 'w') as file:
    file.write('\n'.join(non_empty_lines))

import os
os.remove(input_file_path)
os.remove('result2_tmpSD.txt')


print("completed SD")

Estrazione OX

In [ ]:
#file_in = open("summary.txt", "r")
file_in = open(fn,"r")
outFile = open("result2_tmpOx.txt", "w")

lines = file_in.readlines()

for i, line in enumerate(lines):
    # Just a spacing line
    #line_sp = line.split()
    #print(line_sp)
    line_s = line.split("\t")
    #line_s.index
    #print("line_s1",line_s)
    if line_s:
        try:
            if(line_s[2] == "Comment :"):
                #print("line_s[2]",line_s[2])
                # Keep track of the comment value
                comm = line_s[3]
                # Just a spacing line
                #print("comm",comm)
        except IndexError:
            pass
        #print("line_s[0]",line_s[2])
        #print("line ",line)
        if len(line_s) == 1:
          pass
        else:
          if(line_s[1] == "El."):
            #print("lines[i][:-1] ",lines[i][:-1])
            #print("ID {0} Comment".format(lines[i][:-1]))
            outFile.write("ID {0} Comment\n".format(lines[i][:-1]))
            for j in range(1,loop-1): #loop+1 perchè non estrae un elemento
              #print("{0} {1}\n".format(lines[i+j][:-1], comm), end="")
              outFile.write("{0} {1}\n".format(lines[i+j][:-1], comm))
              #outFile.write(" {0}\n".format(lines))

file_in.close()
outFile.close() #result2.txt

print("completed OX extraction")

In [ ]:
#RESULT3 con spazi e tab corretti OX
newopen = open("result2_finalTmpOx.txt", "w")
with open("result2_tmpOx.txt") as f:
  first_line = f.readline()
  first_line_split = first_line.split()
  first_line = '\t'.join(first_line_split)+'\n'
  newopen.write(first_line)
  #print(first_line)

with open("result2_tmpOx.txt") as f:
  for line in f:
    if 'ID Element	El.' not in line:
      line_split = line.split()
      line = '\t'.join(line_split)+'\n'
      #print(line)
      newopen.write(line)
newopen.close()

input_file_path = 'result2_finalTmpOx.txt'
output_file_path = 'result2_finalOx.txt'

# Read the file and filter out empty lines
with open(input_file_path, 'r') as file:
    lines = file.readlines()
    non_empty_lines = [line.strip() for line in lines if line.strip()]

# Write non-empty lines back to the file
with open(output_file_path, 'w') as file:
    file.write('\n'.join(non_empty_lines))

os.remove(input_file_path)
os.remove('result2_tmpOx.txt')

print("completed OX")

RESULT3.TXT con dati estratti di SD e DL

Extract X-Y coordinates

In [ ]:
# EXTRACT X and Y coodinates
file_in2 = open(fn, "r")
lines = file_in2.readlines()
names = []
Xs = []
Ys = []
Zs = []

for i, line in enumerate(lines):
    line_s = line.split()
    #print(line_s)
    if line_s:
        try:
            if(line_s[4] == "Comment"):
                # Keep track of the comment value
                name = line_s[6]
                names.append(name)
                # Just a spacing line
            elif(line_s[0] == "Stage"):
                # Keep track of the comment value
                x = line_s[3]
                Xs.append(x)
                y = line_s[5]
                Ys.append(y)
                z = line_s[7]
                Zs.append(z)
                # Just a spacing line
        except IndexError:
            pass

import pandas as pd
# Just a spacing line
print("Sample: ",len(names))
print("X: ", len(Xs))
print("Y: ", len(Ys))
print("Z: ", len(Zs))
out = pd.DataFrame(
    {'Sample': names,
     'X':Xs,
     'Y':Ys,
     'Z':Zs
     })
# Just a spacing line
#print(out)
out.to_csv('coordinate.csv', index=False)

COORDINATE.CSV con sample name e coordinate X e Y

TUTTO DA **SCARICARE**